# Docker Swarm

## Init Docker Swarm on Cluster

In [1]:
getent hosts cluster-00.fritz.box | awk '{ print $1 }'

192.168.56.192


In [2]:
docker node ls || true

Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.


In [3]:
docker swarm init --advertise-addr $(getent hosts cluster-00.fritz.box | awk '{ print $1 }')

Swarm initialized: current node (a2qv053ity1k7mpzk1nj9o4is) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-12yr7b2cy4qz7y4nsdk46ymjl69arm4i41nciyparfi284l32v-eswany0pjw2w2noxy9bn8n8yu \
    192.168.56.192:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



In [4]:
JOIN_TOKEN=$(docker swarm join-token -q worker)

In [5]:
# Setup ssh-key for user
ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

localhost | SUCCESS | rc=0 >>
skipped, since /home/pi/.ssh/id_rsa exists


In [6]:
for i in 01 02 03 04; do
    # Remove host key for ip
    ssh-keygen -R cluster-${i}
    # Add ip into known_hosts
    ssh-keyscan -H cluster-${i} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@cluster-${i}
done

# Host cluster-01 found: line 7 type RSA
# Host cluster-01 found: line 8 type ECDSA
# Host cluster-01 found: line 9 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed

/usr/bin/ssh-copy-id: WARNING: All keys were skipped because they already exist on the remote system.

# Host cluster-02 found: line 7 type ECDSA
# Host cluster-02 found: line 8 type RSA
# Host cluster-02 found: line 9 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-02 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-02 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-02 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-co

In [7]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr $(getent hosts cluster-${i}.fritz.box | awk '{ print $1 }') $(getent hosts cluster-00.fritz.box | awk '{ print $1 }'):2377"
done

This node joined a swarm as a worker.
This node joined a swarm as a worker.
This node joined a swarm as a worker.
This node joined a swarm as a worker.


In [8]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
a2qv053ity1k7mpzk1nj9o4is *   cluster-00          Ready               Active              Leader
i2sauj58kyx3czicequ6jg9j9     cluster-01          Ready               Active              
jqfgpge5rrx1b0ch0i5a6ucy7     cluster-03          Ready               Active              
vw2gmehki55or8oc3ekstambg     cluster-02          Ready               Active              
z8pmjnhp58dsdjs4t4adfroxu     cluster-04          Ready               Active              


In [9]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  alexellis2/visualizer-arm

ilfzp37k2yefkep7cpix1e0ce
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


## Visualizer anschauen

In [10]:
echo "http://$(getent hosts cluster-00.fritz.box | awk '{ print $1 }'):8000"

http://192.168.56.192:8000


In [11]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
ilfzp37k2yef        visualizer          replicated          0/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [12]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  stefanscherer/monitor:1.1.0

e64zln5qwqv4oxpo8d5gvsr6s
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [13]:
docker service create --name whoami stefanscherer/whoami:1.1.0

0131abpzjxbmdp2l7puosmp5r
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [14]:
docker service scale whoami=5

whoami scaled to 5


In [15]:
docker service scale whoami=15

whoami scaled to 15


In [16]:
docker service scale whoami=40

whoami scaled to 40


In [17]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [18]:
docker service scale whoami=1

whoami scaled to 1


In [19]:
docker service scale whoami=40

whoami scaled to 40


In [20]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
0131abpzjxbm        whoami              replicated          0/40                stefanscherer/whoami:1.2.0         
e64zln5qwqv4        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
ilfzp37k2yef        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [21]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.1.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [22]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [23]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
0131abpzjxbm        whoami              replicated          21/40               stefanscherer/whoami:1.2.0         
e64zln5qwqv4        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
ilfzp37k2yef        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [24]:
docker service rm whoami

whoami


In [25]:
docker service rm monitor

monitor


In [26]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
a2qv053ity1k7mpzk1nj9o4is *   cluster-00          Ready               Active              Leader
i2sauj58kyx3czicequ6jg9j9     cluster-01          Ready               Active              
jqfgpge5rrx1b0ch0i5a6ucy7     cluster-03          Ready               Active              
vw2gmehki55or8oc3ekstambg     cluster-02          Ready               Active              
z8pmjnhp58dsdjs4t4adfroxu     cluster-04          Ready               Active              


In [27]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

Node left the swarm.
Node left the swarm.
Node left the swarm.
Node left the swarm.


In [28]:
docker swarm leave --force

Node left the swarm.
